# YouTube Caption Orchestrator - Test Notebook

Este notebook demonstra como usar a API Orquestradora para processar vídeos do YouTube.

In [ ]:
# Instala requests
%pip install requests

In [ ]:
import requests
import time
import json

# Configuração
BASE_URL = "http://localhost:8000"
YOUTUBE_URL = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"  # Substitua pela URL desejada

## 1. Health Check

In [ ]:
# Verifica saúde do orquestrador e microserviços
response = requests.get(f"{BASE_URL}/health")
print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

## 2. Iniciar Pipeline

In [ ]:
# Inicia processamento do vídeo
payload = {
    "youtube_url": YOUTUBE_URL,
    "language": "auto",  # ou "pt", "en", etc.
    "remove_noise": True,
    "convert_to_mono": True,
    "sample_rate_16k": True
}

start_time = time.time()
response = requests.post(f"{BASE_URL}/process", json=payload)
elapsed = time.time() - start_time

print(f"Response Time: {elapsed:.3f}s")
print(f"Status Code: {response.status_code}")

result = response.json()
job_id = result["job_id"]

print(f"\nJob ID: {job_id}")
print(json.dumps(result, indent=2))

## 3. Polling do Status

In [ ]:
# Acompanha progresso do pipeline
def check_job_status(job_id, max_attempts=60, interval=5):
    """
    Faz polling do status do job até completar ou falhar
    """
    for attempt in range(max_attempts):
        response = requests.get(f"{BASE_URL}/jobs/{job_id}")
        
        if response.status_code != 200:
            print(f"Erro ao consultar job: {response.status_code}")
            break
        
        data = response.json()
        status = data["status"]
        progress = data["overall_progress"]
        
        # Mostra status dos estágios
        stages = data["stages"]
        print(f"\n[Attempt {attempt + 1}] Status: {status} | Progress: {progress:.1f}%")
        print(f"  Download: {stages['download']['status']} ({stages['download']['progress']:.1f}%)")
        print(f"  Normalization: {stages['normalization']['status']} ({stages['normalization']['progress']:.1f}%)")
        print(f"  Transcription: {stages['transcription']['status']} ({stages['transcription']['progress']:.1f}%)")
        
        # Verifica se completou
        if status in ["completed", "failed", "cancelled"]:
            print(f"\n{'='*60}")
            print(f"Pipeline finalizado com status: {status}")
            print(f"{'='*60}")
            return data
        
        time.sleep(interval)
    
    print("\nTimeout: Pipeline não completou no tempo esperado")
    return None

# Executa polling
final_result = check_job_status(job_id)

## 4. Resultado Final

In [ ]:
# Mostra resultado completo
if final_result:
    print("\n📊 RESULTADO FINAL:")
    print(f"{'='*60}")
    print(f"Status: {final_result['status']}")
    print(f"Progress: {final_result['overall_progress']}%")
    print(f"\n📁 Arquivos:")
    print(f"  Audio: {final_result.get('audio_file', 'N/A')}")
    print(f"  Transcription: {final_result.get('transcription_file', 'N/A')}")
    
    if final_result.get('transcription_text'):
        print(f"\n📝 Transcrição:")
        print(f"{'-'*60}")
        print(final_result['transcription_text'][:500])  # Primeiros 500 caracteres
        if len(final_result['transcription_text']) > 500:
            print("\n[...truncado...]")
    
    if final_result.get('error_message'):
        print(f"\n❌ Erro: {final_result['error_message']}")
    
    print(f"\n⏱️ Tempos:")
    print(f"  Criado: {final_result['created_at']}")
    print(f"  Atualizado: {final_result['updated_at']}")
    print(f"  Completo: {final_result.get('completed_at', 'N/A')}")
else:
    print("❌ Não foi possível obter resultado final")

## 5. Listar Jobs Recentes

In [ ]:
# Lista últimos jobs
response = requests.get(f"{BASE_URL}/jobs?limit=10")
print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

## 6. Estatísticas do Sistema

In [ ]:
# Verifica estatísticas
response = requests.get(f"{BASE_URL}/admin/stats")
print(f"Status Code: {response.status_code}")
print(json.dumps(response.json(), indent=2))